Let's play around with the graphviz backend.

### Shift register

This example is taken from the shift_register notebook.

In [1]:
from magma import *
from magma.primitives import DefineRegister

N = 4
Register4 = DefineRegister(4)
T = Bits(N)

class ShiftRegister(Circuit):
    name = "ShiftRegister"
    IO = ["I", In(T), "O", Out(T), "CLK", In(Bit)]
    @classmethod
    def definition(io):
        regs = [Register4() for _ in range(N)]
        [wire(io.CLK, reg.clk) for reg in regs]  # TODO: Clean up this clock wiring
        wire(io.I, regs[0].D)
        fold(regs, foldargs={"D":"Q"})
        wire(regs[-1].Q, io.O)


Now let's visualize our `ShiftRegister`.

FIXME: ShiftRegister's `._repr_html_` method should allow us to just do `ShiftRegister` instead of the below, but IPython doesn't pick up on it?
It does pick up on the repr in a circuit _instance_ (see the next thing), so is it a metaclass problem?

In [2]:
from IPython.display import HTML
from magma.backend.dot import to_html
HTML(to_html(ShiftRegister))


compiling ShiftRegister ShiftRegister


FIXME: For some separate reason, the visualization is slightly broken (outputs aren't wired up) for a placed circuit / instance (?) like `ShiftRegister()` below.

It's not graphviz-specific, I think: the same issue shows up in firrtl and verilog compile output.

Notice that it does recognize the HTML repr on the circuit instance (not sure why??), so we don't need to do the `to_html` stuff.

In [3]:
ShiftRegister()

compiling  


= ShiftRegister()  # <ipython-input-3-fcb3d66b8664> 1

### Simple ALU

Another example, taken from the Chisel examples folder.

In [4]:
from functools import reduce

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & [cond for _ in range(len(inp))])
    return list(reduce(lambda x, y: x | y, outputs))

class SimpleALU(Circuit):
    name = "SimpleALU"
    IO = ["a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = io.opcode == int2seq(0, 2)
        is_op1 = io.opcode == int2seq(1, 2)
        is_op2 = io.opcode == int2seq(2, 2)
        is_op3 = io.opcode == int2seq(3, 2)
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [5]:
HTML(to_html(SimpleALU))

compiling SimpleALU SimpleALU
